In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils as U
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
import random

### Define the Neural Network

We want to be able to set the weight of the network manually

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, in_dim=2, out_dim=2):
        super(NeuralNet, self).__init__()
        
        self.layer1=nn.Linear(in_features=in_dim, out_features=4)
        self.layer2=nn.Linear(in_features=4, out_features=out_dim)

        self.weights_initialization()
    
    def weights_initialization(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)  # This is the default in PyTorch
                nn.init.constant_(module.bias, 0)

    def forward(self, x):
        out = self.layer1(x)
        out = F.relu(out)
        out = self.layer2(out)
        return out
    
    def get_flat_params(self):
        return U.parameters_to_vector(self.parameters())
    
    def set_flat_params(self, flat_params):
        U.vector_to_parameters(flat_params, self.parameters())

### Load Data

In [3]:
X, y = make_moons(n_samples=1000, noise=0.1, random_state=42)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

### Evolutionary Algorithm

In our case the flat parameters will be the chromosomes

In [ ]:
# Initialize model and get flat parameters
model = NeuralNet()
params_vector = model.get_flat_params()
print(params_vector)


# Configuration
POPULATION_SIZE = 100
CHROMOSOME_LENGTH = len(params_vector)

tensor([ 5.4157e-01, -9.6938e-01,  6.8159e-01, -3.3370e-01,  3.3292e-01,
         6.7682e-01,  9.4453e-01, -1.3159e-01,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  7.5796e-02, -2.5452e-01,  1.1230e-01,
         9.0249e-01, -1.3721e-04, -2.8852e-01,  7.4950e-01,  8.8189e-01,
         0.0000e+00,  0.0000e+00], grad_fn=<CatBackward0>)


In [23]:
def generate_population(size, chromosome_length):
    """ Generate random population """
    return [torch.rand(chromosome_length) for _ in range(size)]

def fitness(chromosome, model):
    """ Algorithm's fitness function """
    model.set_flat_params(chromosome)   # Update model's parameters with the chromosome
    with torch.no_grad():
        outputs = model(X_test)
        pred = outputs.argmax(dim=1)
        accuracy = (pred == y_test).float().mean().item()

    return accuracy

def selection(population, fitness_scores):
    """ Roulette Wheel Selection """
    total_fitness = sum(fitness_scores)
    selection_probs = [f / total_fitness for f in fitness_scores]
    parent1 = population[random.choices(range(len(population)), selection_probs)[0]]
    parent2 = population[random.choices(range(len(population)), selection_probs)[0]]

    return parent1, parent2

def crossover(parent1, parent2):
    """ Two-Point Crossover """
    # Choose two random points
    point1 = random.randint(1, len(parent1)-1)
    point2 = random.randint(1, len(parent1)-1)
    
    # Ensure the points are different
    while point1 == point2:
        point2 = random.randint(1, len(parent1)-1)
    
    if point2 > point1:
        offspring1 = torch.cat(parent1[:point1], parent2[point1:point2], parent2[point2:])
        offspring2 = torch.cat(parent2[:point1], parent1[point1:point2], parent1[point2:])
    else:
        offspring1 = torch.cat(parent1[:point1], parent2[point1:point2], parent2[point2:])
        offspring2 = torch.cat(parent2[:point1], parent1[point1:point2], parent1[point2:])
    
    return offspring1, offspring2

In [ ]:
population = generate_population(POPULATION_SIZE, CHROMOSOME_LENGTH)
fitness_scores = [fitness(individual, model) for individual in population]

print(f"Population Sample: {population[0]}\n")
print(f"Fitness Score: {fitness_scores[0]}\n")
print(f"Selection Result: {selection(population, fitness_scores)}")


Population Sample: tensor([0.3616, 0.8162, 0.8072, 0.3608, 0.1773, 0.0668, 0.8138, 0.1454, 0.4553,
        0.4394, 0.4574, 0.6543, 0.7472, 0.2116, 0.7078, 0.9035, 0.4654, 0.2802,
        0.9302, 0.1145, 0.3001, 0.8633])

Fitness Score: 0.3700000047683716

Selection Result: (tensor([0.6443, 0.9105, 0.3444, 0.6380, 0.7061, 0.6244, 0.5292, 0.4460, 0.6031,
        0.8608, 0.1625, 0.7537, 0.3419, 0.5018, 0.9584, 0.5983, 0.4713, 0.2136,
        0.8183, 0.8074, 0.3154, 0.1748]), tensor([0.3885, 0.9445, 0.3426, 0.4874, 0.0510, 0.3342, 0.8141, 0.5341, 0.0700,
        0.0812, 0.3372, 0.0581, 0.8052, 0.6512, 0.0624, 0.9087, 0.3360, 0.6808,
        0.1081, 0.9445, 0.9932, 0.4115]))
